In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class TtoTClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        """
        입력: (T, N) 형태의 데이터
        출력: (T,) 형태의 스칼라 값들
        
        Args:
            input_dim (int): 입력 특성의 차원 (N)
            hidden_dim (int): 은닉층 차원
        """
        super(TtoTClassifier, self).__init__()
        
        # 각 시점(행)을 독립적으로 처리하는 신경망
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)  # 각 시점마다 하나의 스칼라 값 출력
        )
        
    def forward(self, x):
        """
        Args:
            x: 입력 텐서 [batch_size, T, N] 또는 [T, N]
            
        Returns:
            출력 텐서 [batch_size, T] 또는 [T]
        """
        # 입력이 2D인지 3D인지 확인 (배치 차원 유무)
        original_shape = x.shape
        if len(original_shape) == 2:  # [T, N]
            x = x.unsqueeze(0)  # [1, T, N]

        batch_size, seq_len, feature_dim = x.shape
        
        # 모든 시점을 하나의 큰 배치로 처리
        x_reshaped = x.reshape(-1, feature_dim)  # [batch_size*T, N]
        
        # 각 시점에 대한 예측 (스칼라 값)
        out_flat = self.net(x_reshaped)  # [batch_size*T, 1]
        
        # 원래 형태로 복원
        out = out_flat.reshape(batch_size, seq_len)  # [batch_size, T]
        
        # 원본 입력이 2D였다면 squeeze
        if len(original_shape) == 2:
            out = out.squeeze(0)  # [T]
            
        return out
    
    def classify(self, x, temperature=1.0):
        """
        회귀 출력에 softmax를 적용하여 분류 확률로 변환
        
        Args:
            x: 입력 텐서
            temperature: softmax 온도 매개변수 (낮을수록 더 날카로운 분포)
            
        Returns:
            각 시점에 대한 확률 분포
        """
        logits = self.forward(x)
        
        # softmax 적용
        if len(logits.shape) == 1:  # [T]
            probs = F.softmax(logits / temperature, dim=0)
        else:  # [batch_size, T]
            probs = F.softmax(logits / temperature, dim=1)
            
        return probs
    
# 모델 학습 코드

# 하이퍼파라미터 설정
T = 30  # 시퀀스 길이 (예: T 값의 개수)
N = 3 # 각 시점의 특성 차원 (예: 이미지 특성)
hidden_dim = 32
learning_rate = 0.001
num_epochs = 50
# batch_size = 32

# 모델 초기화
model = TtoTClassifier(N, hidden_dim)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 손실 함수 및 옵티마이저
criterion = nn.MSELoss()  # 회귀 손실
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 데이터 준비
sample_input = torch.rand(T, N).to(device)
# sample_target = torch.rand(T).to(device)

sample_pred = model(sample_input)
sample_class = sample_pred.argmax(dim=0)

print(f"Input shape: {sample_input.shape}")
print(f"Output shape: {sample_pred.shape}")
print(f"Output class: {sample_class}")
